## Test the /search endpoints for auxip and cadip

In [ ]:
# Init environment before running a demo notebook.
from resources.utils import *
init_demo()
from resources.utils import * # reload the global vars again

In [ ]:
# Imports
import itertools
import json
import math
import sys
from dataclasses import dataclass
from urllib.parse import unquote

!pip install iso8601 rfc3339
import iso8601
import rfc3339

In [ ]:
@dataclass
class Error:
    """One error from these tests."""

    endpoint: str
    params: dict
    messages: list[str]

    def __repr__(self):
        sep = "\n  - "
        return (
            f"\nPOST {self.endpoint!r}\n{json.dumps(self.params, indent=2)}\n"
            f"Error(s):{sep}{sep.join(self.messages)}\n"
        )

# List of errors
errors: list[Error] = []

def save_error(*args) -> None:
    """Print and save an error"""
    error = Error(*args)
    print(error, file=sys.stderr)
    errors.append(error)

In [ ]:
# Any collection name to search
adgs_collection = "adgs"
cadip_collection = "cadip"

# Collections to search. If None: search all collections.
collections = None

# Parameters on which we can sort
adgs_sortbys = [
    "id",
    "auxip:id",
    "file:size",
    "type",
    "eviction_datetime",
    "created",
    "start_datetime",
    "end_datetime"
    ]
cadip_sortbys = [
    "id",
    "start_datetime",
    "datetime",
    "end_datetime",
    "published",
    "platform",
    "cadip:id",
    "cadip:num_channels",
    "cadip:station_unit_id",
    "sat:absolute_orbit",
    "cadip:acquisition_id",
    "cadip:antenna_id",
    "cadip:front_end_id",
    "cadip:retransfer",
    "cadip:antenna_status_ok",
    "cadip:front_end_status_ok",
    "cadip:planned_data_start",
    "cadip:planned_data_stop",
    "cadip:downlink_status_ok",
    "cadip:delivery_push_ok"
]
sortbys = None

# auxip or cadip /search endpoint
auxip = False
cadip = False
endpoint = ""

def ignore_search(search_properties=[]):
    """
    Our adgs and cadip station simulators don't handle some parameters so we ignore these calls.
    """
    # "Too complex for adgs sim"
    if auxip:
        if "platform" in search_properties:
            return True
        if search_properties == ("constellation", "product:type"):
            return True

    # Don't ignore, do the call
    return False

In [ ]:
def search_and_check(params={}, from_feature=None, search_properties=[]):
    """
    Call the /search endpoint, check the results, return features and links.

    The search parameters are in the `params` argument.

    If `from_feature` and `search_properties` are set, these search property values 
    are copied from this reference feature.    
    """

    info = f"POST {endpoint!r} with params: {list(search_properties) + list(params.keys())}"

    # Ignore this call
    if ignore_search(search_properties):
        print(f"Ignore {info}")
        return None, None
    print(f"Call {info}")

    # Set the query parameters
    if collections:
        params["collections"] = collections
    for property in search_properties:
        in_value = from_feature.get(property) or from_feature["properties"].get(property)

        if property == "id":
            params["ids"] = [in_value]
            
        elif property == "datetime":
            params["datetime"] = in_value

        # query parameters
        else:
            params.setdefault("query", {})[property] = {"eq": in_value}

    # Format datetime
    try:
        params["datetime"] = params["datetime"].replace("+00:00", "Z")
    except KeyError:
        pass

    # Call the search endpoint, read the returned features
    try:
        response = http_session.post(endpoint, json=params)
    except Exception as exception:
        save_error(endpoint, params, [str(exception)])
        return None, None

    if response.status_code != 200:
        save_error(endpoint, params, [f"Status code: {response.status_code}\n{unquote(response.content)}"])
        return None, None

    ret_features = response.json()["features"]
    if not ret_features:
        save_error(endpoint, params, [f"No features returned"])
        return None, None

    # Check that each returned feature property on which we filtered,
    # has the same value than in the input feature.
    messages: list[str] = []
    for property in search_properties:
        for ret_feature in ret_features:
            in_value = from_feature.get(property) or from_feature["properties"].get(property)
            ret_value = ret_feature.get(property) or ret_feature["properties"].get(property)

            if in_value != ret_value:
                messages.append(f"Wrong {property!r}: {ret_value!r}, expected: {in_value!r}")
                break  # only print error for first wrong feature

    if messages:
        save_error(endpoint, params, messages)
    return ret_features, response.json()["links"]

In [ ]:
#############
# Main code #
#############

# Call auxip and cadip /search endpoint, with one collection # or all collections
for (service, all_collections) in itertools.product(("auxip", "cadip"), (False,)): # True)):

    # For better readability
    auxip = service == "auxip"
    cadip = service == "cadip"

    # Init
    if auxip:
        endpoint = f"{auxip_client.href_adgs}/auxip/search"
        one_collection = adgs_collection
        sortby_date = "created"
        sortbys = adgs_sortbys
    elif cadip:
        endpoint = f"{cadip_client.href_cadip}/cadip/search"
        one_collection = cadip_collection
        sortby_date = "published"
        sortbys = cadip_sortbys

    # Collections to search
    collections = None if all_collections else [one_collection]

    # Get all auxip products or cadip sessions, sorted by newest
    all_features, _ = search_and_check({
        "limit": 10000, 
        "sortby": [{"direction": "desc", "field": sortby_date}]})
    
    if all_features is None:
        continue

    ##################
    # Test filtering #
    ##################

    # We take any existing feature returned by the stations, filter on its properties,
    # and check that the filter was applied.
    feature = all_features[1]

    # All properties on which we can filter
    properties = ["id", "datetime", "platform"]
    if auxip:
        properties += ["constellation", "product:type"]

    # Test all combinations of n properties
    for length in range(1, len(properties) + 1):
        for search_properties in itertools.combinations(properties, length):
            search_and_check({}, feature, search_properties)

    ###################
    # Test pagination #
    ###################

    # Get all auxip products or cadip sessions again, but with a temporal filter: 
    # remove the two most recent features.
    # This is to test the pagination + filtering at the same time.
    date_filter_stop = all_features[1]["properties"]["datetime"].replace("+00:00", "Z")
    date_filter_param = {"datetime": f"1950-01-01T00:00:00Z/{date_filter_stop}"}
    date_filter_features, _ = search_and_check({
        **date_filter_param,
        "limit": 10000, 
        "sortby": [{"direction": "desc", "field": sortby_date}]})

    # Test all sortby fields, with and without reverse, with and without a date filter
    for (
        sortby, 
        reverse, 
        reference_features
    ) in itertools.product(
        sortbys, 
        (False, True), 
        (all_features, date_filter_features)
    ):
        if not reference_features:
            continue
        
        # Split the total number of features in n pages.
        # 3 pages for the 1st sortby, a single one (this is faster) for the next
        pages = 3 if sortby == sortbys[0] else 1
        limit = math.ceil(len(reference_features) / pages)

        # Endpoint parameters
        params = {
            "limit": limit,
            "sortby": [{"direction": "desc" if reverse else "asc", "field": sortby}]
        }
        if reference_features == date_filter_features:
            params.update(date_filter_param) # add filtering on date
        page_params = params # will be update for next pages with a token

        def sorted_value(feature: dict):
            """
            If we sort by e.g. the datetime, return the input feature datetime. 
            It is either in the feature root, in its properties, or in the first asset.
            """
            try:
                ret = (
                    feature.get(sortby, None) or 
                    feature["properties"].get(sortby, None) or
                    list(feature["assets"].values())[0].get(sortby, None))
                if ret is None:
                    raise KeyError
                return ret
            except (KeyError, IndexError):
                raise RuntimeError(f"Feature is missing field {sortby!r}:\n{json.dumps(feature, indent=2)}")

        # Concatenate all values for the current sortby field 
        # that will be returned by the paginated and sorted endpoints
        all_paginated_values = []

        # The result should be the same as when we manually sort the values 
        # returned by the non-paginated endpoint.
        try:
            all_expected_values = sorted(
                [sorted_value(feature) for feature in reference_features],
                reverse=reverse)
        except RuntimeError as error:
            save_error(endpoint, page_params, [str(error)])
            continue # try next sortby field

        test_results = True

        # For each page to request
        for page in range(pages):

            # Request current page
            paginated_features, links = search_and_check(page_params)

            # In case of error, don't process next pages
            if not links:
                test_results = False
                break

            # Read the previous (for page > 1) and next pages token = 
            # fetch the link which rel=="previous" or "next", and read its body/token value
            try:
                x_page_token = lambda x: \
                    [link for link in links if link["rel"] == x][0]["body"]["token"]
                next_page_token = x_page_token("next")
                prev_page_token = x_page_token("previous") if page else None
            except (KeyError, IndexError):
                save_error(endpoint, page_params, [f"Missing self/previous/next link(s)"])
                test_results = False
                break # don't process next page

            # Check the token for next and previous page
            if f"page={page+2}" not in next_page_token:
                save_error(endpoint, page_params, 
                    [f"Wrong 'next' page token: {next_page_token!r}, should contain: 'page={page+2}'"])
                test_results = False
                break # don't process next page
            if prev_page_token and (f"page={page}" not in prev_page_token):
                save_error(endpoint, page_params, 
                    [f"Wrong 'previous' page token: {prev_page_token!r}, should contain: 'page={page}'"])
                
            # Use the "next" token when requesting the next page
            page_params.update({"token": next_page_token})

            # Concatenate all values for the current sortby field 
            # returned by the paginated and sorted endpoint
            try:
                all_paginated_values += [sorted_value(feature) for feature in paginated_features]
            except RuntimeError as error:
                save_error(endpoint, page_params, [str(error)])
                break # don't process next page

        # After requesting all the pages, check that the results are sorted as expected
        if test_results and (all_paginated_values != all_expected_values):
            save_error(endpoint, page_params, [
                f"Paginated values (len={len(all_paginated_values)}) "
                f"are not as expected (len={len(all_expected_values)}):\n" 
                f"Got:      {all_paginated_values}\n"
                f"Expected: {all_expected_values}"])

    #############################
    # Test the datetime formats #
    #############################

    # Get distinct datetimes from all features
    all_datetimes = sorted([
        iso8601.parse_date(feature["properties"]["datetime"]) 
        for feature in all_features])
    assert(len(all_datetimes) >= 5)

    # Search by fixed datetime
    fixed = all_datetimes[1]
    fixed_str = rfc3339.rfc3339(fixed)

    # Search by closed interval. 
    # The min and max datetimes should not be returned by the http request.
    closed_str = f"{rfc3339.rfc3339(all_datetimes[1])}/{rfc3339.rfc3339(all_datetimes[4])}"
    closed = all_datetimes[2:4]

    # Search by from interval.  
    # The min datetime should not be returned by the http request.
    from_str = f"{rfc3339.rfc3339(all_datetimes[-3])}/.."
    from_ = all_datetimes[-2:]

    # Search by to interval.  
    # The max datetime should not be returned by the http request.
    to_str = f"../{rfc3339.rfc3339(all_datetimes[2])}"
    to_ = all_datetimes[:2]

    # Search features with the given datetime, return only the datetimes
    def search_datetime(datetime_param: dict) -> tuple[list[datetime], list[str]]:
        features, _ = search_and_check(datetime_param)
        features = features or []
        return sorted([
            iso8601.parse_date(feature["properties"]["datetime"]) 
            for feature in features])
    
    # Convert datetime list to str for printing
    def dt_to_str(datetimes: list[datetime]) -> str:
        return "\n    - " + "\n    - ".join([rfc3339.rfc3339(d) for d in datetimes])

    # Search with a fixed datetime
    datetime_param = {"datetime": fixed_str}
    datetimes = search_datetime(datetime_param)
    if (len(datetimes) != 1) or (datetimes[0] != fixed):
        save_error(endpoint, datetime_param, [
            f"Wrong datetime(s): {dt_to_str(datetimes)}\n    Expected: {dt_to_str([fixed])}"])
        
    # Search with a closed interval
    datetime_param = {"datetime": closed_str}
    datetimes = search_datetime(datetime_param)
    if datetimes != closed:
        save_error(endpoint, datetime_param, [
            f"Wrong datetime(s): {dt_to_str(datetimes)}\n    Expected: {dt_to_str(closed)}"])
        
    # Search with a from interval
    datetime_param = {"datetime": from_str}
    datetimes = search_datetime(datetime_param)
    if datetimes != from_:
        save_error(endpoint, datetime_param, [
            f"Wrong datetime(s): {dt_to_str(datetimes)}\n    Expected: {dt_to_str(from_)}"])
        
    # Search with a to interval
    datetime_param = {"datetime": to_str}
    datetimes = search_datetime(datetime_param)
    if datetimes != to_:
        save_error(endpoint, datetime_param, [
            f"Wrong datetime(s): {dt_to_str(datetimes)}\n    Expected: {dt_to_str(to_)}"])

In [ ]:
# Print again all errors at the end, exit the notebook with an error
if errors:
    message = "\n## Error message start ##\n"
    for error in errors:
        message += str(error)
    message += "\n## Error message finish ##\n"
    raise RuntimeError(message)